In [1]:
import tensorflow as tf
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.layers import Dense

# DNN model definition
class Model(tf.keras.Model):
    def __init__(self):
        super(Model, self).__init__()
        self.input_layer  = Dense(256, activation='relu', input_shape=(784,))
        self.hidden_layer = Dense(128, activation='relu')
        self.output_layer = Dense(10, activation='softmax')
    
    def call(self, x):
        x = self.input_layer(x)
        x = self.hidden_layer(x)
        output = self.output_layer(x)
        return output

2024-10-26 15:24:54.074953: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1729970694.094157 2840663 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1729970694.100690 2840663 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-10-26 15:24:54.122252: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [2]:
if __name__ == "__main__":
    
    # model, cost function, optimizer
    model = Model()
    cross_entropy = tf.keras.losses.CategoricalCrossentropy(from_logits=False)
    optimizer = Adam(1e-4)
    
    # load data
    mnist = tf.keras.datasets.fashion_mnist
    (x_train, y_train), (x_test, y_test) = mnist.load_data()
    # data preprocessing
    x_train, x_test = x_train / 255.0, x_test / 255.0
    
    # minibatch size
    batch_size = 32
    num_train_data = x_train.shape[0]
    num_test_data = x_test.shape[0]
    
    # training
    num_epoch = 10
    for e in range(num_epoch):
        for i in range(num_train_data // batch_size):
            x_batch = x_train[i * batch_size : (i + 1) * batch_size]
            y_batch = y_train[i * batch_size : (i + 1) * batch_size] 
        
            x_batch = x_batch.reshape(-1, 28 * 28)
            y_batch = tf.one_hot(y_batch, 10)
        
            with tf.GradientTape() as tape:
                # prediction
                predicts = model(x_batch)
                #
                model_params = model.trainable_variables
                # calculate loss
                losses = cross_entropy(predicts, y_batch)
            # compute gradient
            grads = tape.gradient(losses, model_params)
            # model update
            optimizer.apply_gradients(zip(grads, model_params))

/gpfs/projects/KeeGroup/ykee/myenv/REINFORCE/lib/python3.12/site-packages/keras/src/layers/core/dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)
I0000 00:00:1729970717.966015 2840663 gpu_device.cc:2022] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 38487 MB memory:  -> device: 0, name: NVIDIA A100-PCIE-40GB, pci bus id: 0000:21:00.0, compute capability: 8.0
I0000 00:00:1729970717.968800 2840663 gpu_device.cc:2022] Created device /job:localhost/replica:0/task:0/device:GPU:1 with 38487 MB memory:  -> device: 1, name: NVIDIA A100-PCIE-40GB, pci bus id: 0000:81:00.0, compute capability: 8.0


In [5]:
import numpy as np
# testing
# Convert y_test to one-hot if needed
y_test_one_hot = tf.one_hot(y_test, 10)

# Make predictions
predictions = model(x_test.reshape(-1, 28 * 28), training=False)  # Set training=False for inference mode

# Calculate accuracy
correct_predictions = tf.equal(tf.argmax(predictions, axis=1), tf.argmax(y_test_one_hot, axis=1))
accuracy = tf.reduce_mean(tf.cast(correct_predictions, tf.float32))

print("Test Accuracy:", accuracy.numpy())

# Optional: Print predictions for the first 5 test samples for manual verification
print("Predictions:", np.argmax(predictions[:5], axis=1))
print("True Labels:", y_test[:5])

Test Accuracy: 0.8598
Predictions: [9 2 1 1 6]
True Labels: [9 2 1 1 6]
